# Traffic Lights Dataset

In [15]:
##### Load in the data 
import pandas as pd
audio = pd.read_csv('AKUSTISCHEAMPELOGD.csv')
noaudio =  pd.read_csv('AMPELOGD.csv')

import folium
audio['longitude'] = audio['SHAPE'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')[0].astype(float)
audio['latitude'] = audio['SHAPE'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')[1].astype(float)

noaudio['longitude'] = noaudio['SHAPE'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')[0].astype(float)
noaudio['latitude'] = noaudio['SHAPE'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')[1].astype(float)

In [ ]:
##### Create simple map with traffic light information
vienna_map = folium.Map(location=[48.2082, 16.3738], zoom_start=12)

### Add points from the 'audio' dataset (green)
for _, row in audio.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='green',
        fill=True,
        fill_opacity=0.7,
        popup=row.get('KREUZUNG', 'Audio Point')
    ).add_to(vienna_map)

### Add points from the 'noaudio' dataset (red)
for _, row in noaudio.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=row.get('KREUZUNG', 'No Audio Point') 
    ).add_to(vienna_map)

### Save the map as an HTML file
vienna_map.save('vienna_audio_noaudio_map.html')

print("Map created! Open 'vienna_audio_noaudio_map.html' to view it.")

In [23]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim

In [24]:
##### Prepare for path 
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim

### Get walking map 
graph = ox.graph_from_place("Vienna, Austria", network_type="walk")
geolocator = Nominatim(user_agent="walking_route_planner")

### Get start and end point
start_location = geolocator.geocode("Fürstengasse 1, Vienna, Austria") # here you can input your starting point
end_location = geolocator.geocode("Schlickplatz, Vienna, Austria") # here you can input your destination 
point_a = (start_location.latitude, start_location.longitude)
point_b = (end_location.latitude, end_location.longitude)

In [ ]:
##### Get route with and without taking audio streetlights into consideration 
no_audio_points = list(zip(noaudio['latitude'], noaudio['longitude']))

### find start and end node 
start_node = ox.distance.nearest_nodes(graph, point_a[1], point_a[0])
end_node = ox.distance.nearest_nodes(graph, point_b[1], point_b[0])
restricted_nodes = [
    ox.distance.nearest_nodes(graph, point[1], point[0]) for point in no_audio_points
]

### restricted graph
graph_with_restrictions = graph.copy()
for node in restricted_nodes:
    if node in graph_with_restrictions:
        graph_with_restrictions.remove_node(node)

### calculate routes 
try:
    # Route avoiding "no audio lights"
    restricted_route = nx.shortest_path(graph_with_restrictions, start_node, end_node, weight="length")
    print("Route avoiding no audio lights found!")
except nx.NetworkXNoPath:
    print("No route found avoiding no audio lights.")
    restricted_route = None

try:
    # Direct route without avoiding "no audio lights"
    direct_route = nx.shortest_path(graph, start_node, end_node, weight="length")
    print("Direct route found!")
except nx.NetworkXNoPath:
    print("No direct route found.")
    direct_route = None

In [ ]:
##### Implement routes into interactive map
vienna_map = folium.Map(location=[48.2082, 16.3738], zoom_start=14)

for _, row in audio.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='green',
        fill=True,
        fill_opacity=0.7,
        popup=row.get('KREUZUNG', 'Audio Traffic Light')
    ).add_to(vienna_map)

for _, row in noaudio.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=row.get('KREUZUNG', 'No Audio Traffic Light')
    ).add_to(vienna_map)

def add_route_to_map(route, graph, map_obj, color="blue", label="Route"):
    if route is not None:
        # Convert route node IDs to lat/lon
        route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
        # Add the route as a PolyLine
        folium.PolyLine(
            route_coords, color=color, weight=5, opacity=0.8, tooltip=label
        ).add_to(map_obj)

add_route_to_map(direct_route, graph, vienna_map, color="red", label="Direct Route")
add_route_to_map(restricted_route, graph_with_restrictions, vienna_map, color="green", label="Restricted Route (Avoiding No Audio Lights)")

folium.Marker(
    location=point_a,
    popup="Start: Palais Liechtenstein",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(vienna_map)

folium.Marker(
    location=point_b,
    popup="End: Rossauer Kaserne",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(vienna_map)

vienna_map.save('vienna_routes_with_traffic_lights_map.html')
print("Interactive map with traffic lights and routes saved as 'vienna_routes_with_traffic_lights_map.html'.")